In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [2]:
!pip install progressbar2
from progressbar import progressbar

## Scraping

### Scraping Functions

In [3]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [4]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [5]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [6]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [7]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:23 Time:  0:00:23
100% (40 of 40) |########################| Elapsed Time: 0:00:23 Time:  0:00:23


In [8]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:24 Time:  0:00:24
100% (40 of 40) |########################| Elapsed Time: 0:00:23 Time:  0:00:23


In [9]:
nbatest.shape

(785, 3)

In [10]:
nfltest.shape

(889, 3)

##### These scrape_add functions add to already built csvs

In [ ]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [11]:
nfltest.head(10)

,subreddit,title,selftext
t3_fo7ihs,nfl,Talko Tuesday,"Welcome to today's open thread, where /r/nfl u..."
t3_fo9ztd,nfl,AMA with Ravens FB/DL Pat Ricard!,Check it out here: [https://www.reddit.com/r/...
t3_fof7od,nfl,[Sessler] Daily reminder that SAM BRADFORD mad...,
t3_foouha,nfl,[Schefter] Rodger Goodell has ordered all NFL ...,
t3_fob82s,nfl,[Panthers] #Panthers have released QB Cam Newton,
t3_fo7hft,nfl,[Mays] Glad the Bears nabbed Nick Foles while ...,
t3_foq2ka,nfl,[Freeman] One thing to pass along: I’m increas...,
t3_fop5nx,nfl,[OC] [Highlight] [Longform] Fans pushing and s...,
t3_fod4n2,nfl,"In Honor of Cam being Released, Here's One of ...",
t3_fo9id8,nfl,[Sherman] do you guys ever think about how in ...,


In [13]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [14]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

KeyError: "['selftext'] not found in axis"

In [15]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [17]:
train.tail(10)

,subreddit,title
t3_fnb2e8,nba,Darrell Armstrong beats the buzzer at the 1996 Slam Dunk Contest
t3_fn9n03,nba,Paul George takes Lance Stephenson to Moves.com to beat the buzzer. Westbrook responds with Lance's air guitar celebration.
t3_fnnjbp,nba,"Imagine if you had a Jeff Bezos level net worth, how much do you think you would have to pay LeBron to lease him as your vehicle, so that he could give you piggy back rides everywhere?"
t3_fnauwo,nba,Reasons why the 2011-2012 NBA season was one of the most eventful seasons in NBA history.
t3_fnd6b9,nba,Dwade vs KD age 23-30
t3_fmzfs9,nba,"Steve Blake Dishes Out NBA Record 14 Assists in a Quarter (February 22, 2009)"
t3_fmnhan,nba,Best game from every NBA team starters + sixth man this season
t3_fn0i95,nba,Dennis Rodman dives for the rebound in a full horizontal extent
t3_fn4krg,nba,The New York Knicks get absolutely obliterated by the sheer greatness of TJ “Michael Jeffrey Jordan” McConnell after Porzingis airballs the corner 3 like a dumbass
t3_fnp0ex,nba,Would you take prime Lebron or Andrew Wiggins with one leg and a gun?


In [19]:
train.shape

(1674, 2)

##### Tokenize (grab only word characters)

In [20]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [24]:
print('Hello\nWorld')

Hello
World


In [22]:
print(r'Hello\nWorld')

Hello\nWorld


In [25]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [26]:
train.head()

,subreddit,title
t3_fo7ihs,nfl,"[talko, tuesday]"
t3_fo9ztd,nfl,"[ama, with, ravens, fb, dl, pat, ricard]"
t3_fof7od,nfl,"[sessler, daily, reminder, that, sam, bradford, made, 129, 982, 500, to, go, 34, 48, 1, never, made, a, pro, bowl, or, all, pro, team, never, started, a, playoff, game, very, possibly, lives, in, a, castle, right, now, off, the, coast, of, some, majestic, ocean]"
t3_foouha,nfl,"[schefter, rodger, goodell, has, ordered, all, nfl, teams, to, shut, down, their, training, facilities, at, 6, p, m, local, time, today, due, to, the, covid, 19, virus, nfl, will, reassess, on, april, 8, when, they, can, open, again]"
t3_fob82s,nfl,"[panthers, panthers, have, released, qb, cam, newton]"


In [27]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [29]:
train.head()

,subreddit,title
t3_fo7ihs,nfl,talko tuesday
t3_fo9ztd,nfl,ama with ravens fb dl pat ricard
t3_fof7od,nfl,sessler daily reminder that sam bradford made 129 982 500 to go 34 48 1 never made a pro bowl or all pro team never started a playoff game very possibly lives in a castle right now off the coast of some majestic ocean
t3_foouha,nfl,schefter rodger goodell has ordered all nfl teams to shut down their training facilities at 6 p m local time today due to the covid 19 virus nfl will reassess on april 8 when they can open again
t3_fob82s,nfl,panthers panthers have released qb cam newton


### Train test split and converting series to list of strings then to array

In [30]:
X = train[['title']]
y = train['subreddit']

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [32]:
# baseline is

y.value_counts(normalize=True)

nfl    0.531063
nba    0.468937
Name: subreddit, dtype: float64

In [33]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [34]:
len(clean_train_data)

1255

In [35]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [36]:
len(clean_test_data)

419

In [37]:
clean_train_data

['was steve nash the best player on the suns during his mvp years',
 'keenan allen in reply to mike williams aye no cap you the best deep threat in the league facts',
 'pelissero te seth devalve has agreed to terms with the panthers per his agent chriscabott of steinbergsports',
 'how often do players mess up alley oops in real games',
 'david griffin says league is focused on resuming not canceling season',
 'charles barkley blocks kenny the jet smith in the final seconds to secure a suns victory',
 'mystery ball theatre 3000',
 'who is your early pick for worst team in football for 2020 and why',
 'fiorio hearing some chatter that jadeveon clowney and the seahawks are moving toward a new deal',
 'lebron james 4th quarter stats from 2011 nba finals',
 'wilson bengals signed wide receiver michael thomas rams southern miss',
 'ticket refunds',
 'qbs at line of scrimmage including the famous omaha',
 'rapaport the bears signed ss jordan lucas to a 1 year deal worth 1m as the former chief

### Count Vectorizer

In [38]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [39]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [40]:
# convert to array

train_data_features = train_data_features.toarray()

In [42]:
# check shapes

train_data_features.shape, test_data_features.shape

((1255, 5000), (419, 5000))

In [43]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [44]:
vocab[0:200]

['000',
 '00pm',
 '03',
 '03 19',
 '05m',
 '07',
 '10',
 '10 5m',
 '10 highest',
 '10 highest paid',
 '10 million',
 '10 players',
 '10 years',
 '100',
 '100 picks',
 '11',
 '114',
 '12',
 '13',
 '13 5m',
 '14',
 '14 assists',
 '14 assists quarter',
 '15',
 '15 rebounds',
 '150',
 '150 000',
 '15m',
 '15th',
 '16',
 '16m',
 '17',
 '17 game',
 '17 million',
 '18',
 '18 4m',
 '180',
 '19',
 '19 relief',
 '1983',
 '1986',
 '1988',
 '1991',
 '1992',
 '1993',
 '1994',
 '1996',
 '1997',
 '1998',
 '1m',
 '1m deal',
 '1st',
 '1st franchise',
 '1st franchise history',
 '1st round',
 '1st team',
 '1st team pro',
 '20',
 '20 million',
 '20 nba',
 '20 nba season',
 '20 percent',
 '20 season',
 '20 seconds',
 '20 years',
 '2000',
 '2000 present',
 '2000s',
 '2001',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2009',
 '2010',
 '2010s',
 '2011',
 '2012',
 '2013',
 '2013 nba',
 '2014',
 '2015',
 '2015 nfl',
 '2015 nfl draft',
 '2016',
 '2016 finals',
 '2017',
 '2017 2018',
 '2017 playoffs',
 '2017 p

## MODELING

### Logistic Regression

In [45]:
from sklearn.linear_model import LogisticRegression

In [46]:
# fit logistic regression model

lr = LogisticRegression(penalty='l2')

In [47]:
# shape check

train_data_features.shape, y_train.shape

((1255, 5000), (1255,))

In [48]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
lr.score(train_data_features, y_train)

0.9920318725099602

In [50]:
lr.score(test_data_features, y_test)

0.9045346062052506

### Feature comparison

Creates a dataframe that matches features to coefficients

In [51]:
coef_list = lr.coef_.tolist()

In [52]:
coef_list = coef_list[0]

In [53]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [59]:
coef_df.sort_values(by = ['coefs']).tail(50)

,features,coefs
234,agent,0.639090
364,bears,0.642491
431,bowl,0.648901
2501,patricia,0.650753
2227,million,0.652544
1091,gm,0.669120
2519,people,0.678183
637,colts,0.679068
870,eagles,0.686236
4706,thank,0.697081


### Let's throw out these unfair words and rerun

In [60]:
sw = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

sw.update(extra_stopwords)

In [61]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = sw,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1255, 5000), (419, 5000))

In [62]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [63]:
lr.score(train_data_features, y_train)

0.9896414342629483

In [64]:
lr.score(test_data_features, y_test)

0.8591885441527446

In [ ]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

### Decision Tree

In [65]:
from sklearn.tree import DecisionTreeClassifier

In [66]:
tree = DecisionTreeClassifier()

In [67]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [68]:
tree.score(train_data_features, y_train)

0.998406374501992

In [69]:
tree.score(test_data_features, y_test)

0.7995226730310262

### Random Forest

In [70]:
from sklearn.ensemble import RandomForestClassifier

In [75]:
forest = RandomForestClassifier(n_estimators = 1000)

In [76]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [77]:
forest.score(train_data_features, y_train)

0.998406374501992

In [78]:
forest.score(test_data_features, y_test)

0.8424821002386634

###  Matrix on Logistic Regression

In [79]:
from sklearn.metrics import confusion_matrix

In [80]:
y_pred = lr.predict(test_data_features)

In [81]:
cm = confusion_matrix(y_test, y_pred)

In [82]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [83]:
cm_df

,predict_neg,predict_pos
actual_neg,173,23
actual_pos,36,187


## Checking where our model failed

In [84]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [85]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [86]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [87]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [88]:
mismatches

,y_actual,y_predicted,title
t3_fkz992,nfl,nba,michael silver plan is for stidham to start next season
t3_flddeb,nfl,nba,happy birthday andy reid
t3_fle9mn,nfl,nba,ochocinco jeffokudah you ready to be a lion
t3_flg90k,nfl,nba,nfl network top 10 quarterback duels
t3_flhays,nfl,nba,seahawks kicking the tires on multiple veteran pass rushers
t3_flxgjm,nfl,nba,which qb would you least like belichick to have
t3_flxug7,nfl,nba,can a receiver tackle or contact an opposing player while in possession on purpose
t3_fly8mb,nfl,nba,salary cap info
t3_fm0lxo,nfl,nba,finn espn will air in full the academy award winning 30 for 30 documentary o j made in america over four nights in primetime from march 23 26 begins at 7 p m each night no exaggeration might be the best thing i ve ever watched on television
t3_fm83u3,nfl,nba,besides kyler murray who are some of the best multi sport athletes in the nfl


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [89]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=sw,
                            max_features=5000,
                            ngram_range=(1, 3))

In [90]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1255, 5000), (419, 5000))

In [91]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [92]:
lr.score(train_data_features, y_train)

0.9816733067729083

In [93]:
lr.score(test_data_features, y_test)

0.8758949880668258

### Let's try on some other subreddits

In [94]:
train = pd.concat([politics_test, conservative_test])

In [95]:
X = train[['title']]
y = train['subreddit']

In [96]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [98]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [99]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [100]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

In [101]:
vocab

['000',
 '000 masks',
 '000 student',
 '000 student debt',
 '03',
 '11',
 '12',
 '130',
 '130 000',
 '14',
 '14 day',
 '15',
 '15 day',
 '19',
 '19 crisis',
 '19 deaths',
 '19 lies',
 '19 mississippi',
 '19 mississippi going',
 '19 outbreak',
 '19 pandemic',
 '19 zero',
 '19 zero new',
 '20',
 '20 years',
 '2020',
 '2020 live',
 '2020 live 30pm',
 '2021',
 '21',
 '23',
 '23rd',
 '25',
 '25 years',
 '2nd',
 '2nd amendment',
 '30',
 '30 000',
 '30 000 student',
 '300',
 '300 deputies',
 '300 deputies patrol',
 '30pm',
 '30pm edt',
 '450',
 '450 progressive',
 '450 progressive organizations',
 '46',
 '50',
 '50 000',
 '500',
 '500 billion',
 '500 billion corporate',
 '55',
 '55 americans',
 '55 americans approve',
 '60',
 '600',
 '600 week',
 '70',
 '95',
 '99',
 'abandon',
 'abortion',
 'abortion essential',
 'abortion rights',
 'abortion rights groups',
 'abortions',
 'abortions coronavirus',
 'abortions coronavirus outbreak',
 'absolutely',
 'accountable',
 'accused',
 'accuses',
 'aco

### Modeling

In [102]:
lr = LogisticRegression(penalty = 'l2')

In [103]:
train_data_features.shape, y_train.shape

((1365, 5000), (1365,))

In [104]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9794871794871794

In [105]:
lr.score(test_data_features, y_test)

0.743421052631579

In [106]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [107]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
1637,media,-1.593190
381,china,-1.592705
2909,president trump,-1.294702
738,dems,-1.193090
392,chinese,-1.107759
...,...,...
2311,nra,1.007092
4797,unemployment,1.141411
2219,need,1.215471
1465,mail,1.448735


In [111]:
coef_df.loc[coef_df.features.str.contains('45')]

,features,coefs
44,450,0.064635
45,450 progressive,0.064635
46,450 progressive organizations,0.064635
1313,join 450,0.064635
1314,join 450 progressive,0.064635
1369,leaders join 450,0.064635


In [110]:
coef_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
features    5000 non-null object
coefs       5000 non-null float64
dtypes: float64(1), object(1)
memory usage: 78.2+ KB


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [112]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [113]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [114]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [115]:
import requests

In [116]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [117]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [118]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [119]:
colors['olive']

(110, 117, 14)

In [120]:
colors['red']

(229, 0, 0)

In [121]:
colors['black']

(0, 0, 0)

In [122]:
colors.keys()

dict_keys(['cloudy blue', 'dark pastel green', 'dust', 'electric lime', 'fresh green', 'light eggplant', 'nasty green', 'really light blue', 'tea', 'warm purple', 'yellowish tan', 'cement', 'dark grass green', 'dusty teal', 'grey teal', 'macaroni and cheese', 'pinkish tan', 'spruce', 'strong blue', 'toxic green', 'windows blue', 'blue blue', 'blue with a hint of purple', 'booger', 'bright sea green', 'dark green blue', 'deep turquoise', 'green teal', 'strong pink', 'bland', 'deep aqua', 'lavender pink', 'light moss green', 'light seafoam green', 'olive yellow', 'pig pink', 'deep lilac', 'desert', 'dusty lavender', 'purpley grey', 'purply', 'candy pink', 'light pastel green', 'boring green', 'kiwi green', 'light grey green', 'orange pink', 'tea green', 'very light brown', 'egg shell', 'eggplant purple', 'powder pink', 'reddish grey', 'baby shit brown', 'liliac', 'stormy blue', 'ugly brown', 'custard', 'darkish pink', 'deep brown', 'greenish beige', 'manilla', 'off blue', 'battleship gre

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [123]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1], [5, 2])

5.0990195135927845

In [126]:
distance([1,2,3], [4,5,6])

5.196152422706632

The `subtractv` function subtracts one vector from another:

In [127]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [128]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [129]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [130]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

True

In [131]:
print(distance(colors['red'], colors['green']))
print(distance(colors['red'], colors['pink']))

273.70787347096905
232.76812496559748


### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [132]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [133]:
closest(colors, colors['red'])

['red',
 'fire engine red',
 'bright red',
 'tomato red',
 'cherry red',
 'scarlet',
 'vermillion',
 'orangish red',
 'cherry',
 'lipstick red']

... or the ten colors closest to (150, 60, 150):

In [136]:
closest(colors, [1000, 1000, 1000])

['white',
 'pale grey',
 'very light pink',
 'off white',
 'ice blue',
 'very pale blue',
 'very light blue',
 'really light blue',
 'eggshell',
 'ice']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [142]:
subtractv(colors['black'], colors['white'])

[-255, -255, -255]

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [143]:
closest(colors, addv(colors['blue'], colors['green']))

['bright turquoise',
 'bright light blue',
 'bright aqua',
 'cyan',
 'neon blue',
 'aqua blue',
 'bright cyan',
 'bright sky blue',
 'aqua',
 'bright teal']

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [144]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

['medium grey',
 'purple grey',
 'steel grey',
 'battleship grey',
 'grey purple',
 'purplish grey',
 'greyish purple',
 'steel',
 'warm grey',
 'green grey']

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [145]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['pink'], colors['red'])
closest(colors, addv(pink_to_red, colors['blue']))

['neon blue',
 'bright sky blue',
 'bright light blue',
 'cyan',
 'bright cyan',
 'bright turquoise',
 'clear blue',
 'azure',
 'dodger blue',
 'lightish blue']

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [146]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

['true green',
 'dark grass green',
 'grassy green',
 'racing green',
 'forest',
 'bottle green',
 'dark olive green',
 'darkgreen',
 'forrest green',
 'grass green']

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [149]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

Roses are red, violets are blue
Roses are bright red, violets are bright blue
Roses are vermillion, violets are cerulean
Roses are orangish red, violets are medium blue
Roses are orangered, violets are french blue
Roses are red orange, violets are windows blue
Roses are bright orange, violets are dirty blue
Roses are blood orange, violets are flat blue
Roses are orangered, violets are dirty blue
Roses are red orange, violets are dull blue
Roses are bright orange, violets are muted blue
Roses are orange red, violets are denim
Roses are tomato, violets are dusk blue
Roses are orangish red, violets are denim blue


### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [154]:
import spacy
# this is the normal way to do it, but doesn't work on my computer
# !python -m spacy download en_core_web_sm
# nlp = spacy.load('en_core_web_sm')

# so I have to do it this way
# !pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz
import en_core_web_sm
nlp = en_core_web_sm.load()

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [155]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

[147.44839067702551, 113.65371809100999, 100.13540510543841]


Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [156]:
closest(colors, avg_color)

['reddish grey',
 'brownish grey',
 'brownish',
 'brown grey',
 'mocha',
 'grey brown',
 'puce',
 'dull brown',
 'pinkish brown',
 'dark taupe']

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [157]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

['sickly yellow',
 'piss yellow',
 'puke yellow',
 'vomit yellow',
 'dirty yellow',
 'mustard yellow',
 'dark yellow',
 'olive yellow',
 'macaroni and cheese',
 'pea']

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [158]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

mauve trousers
dusty rose trousers
dusky rose trousers
brownish pink trousers
old pink trousers
reddish grey trousers
dirty pink trousers
old rose trousers
light plum trousers
ugly pink trousers


That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [159]:
dracula[560]

hour

In [160]:
_.vector

array([-3.3890932 ,  3.8036323 , -2.6519728 ,  1.6794462 , -0.03245598,
        0.34949422,  1.4263201 ,  0.7514826 ,  1.5714422 ,  6.836588  ,
       -0.87871516, -2.5962636 ,  4.4980946 , -3.2636712 ,  0.59583217,
        1.7193251 , -1.7698543 , -2.6230168 ,  0.32886952, -1.9750781 ,
        3.1977305 , -2.3372014 , -2.340062  , -2.59442   , -1.707979  ,
        1.4709201 ,  0.46112838, -4.2141914 ,  3.8767452 ,  1.9445885 ,
        2.5187936 ,  2.709597  ,  4.9126153 , -2.4080238 ,  1.2024103 ,
       -1.7131364 ,  2.7877707 , -4.4838257 , -1.9192042 , -2.094125  ,
        3.3191376 ,  2.954278  , -1.6426673 , -2.4524295 , -0.97719395,
        1.2865635 , -0.9342208 , -0.07476962, -1.0927347 ,  0.39212835,
        2.6349268 , -3.4252431 ,  0.7364484 , -0.04758237, -0.8591391 ,
        0.5237043 ,  3.3768516 , -0.6579486 , -0.34316725, -0.7823252 ,
        2.7190056 , -2.2406356 , -1.2399366 , -0.10923919,  0.38356325,
       -0.07189876,  4.31141   ,  2.5327733 ,  0.69408464, -0.58

In [161]:
# previously we've used the _sm model, which doesn't include all vectors.
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
# almost 800mb!
import en_core_web_lg
nlp = en_core_web_lg.load()

In [162]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [163]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [164]:
nlp.vocab['alligator'].vector

array([-4.9304e-01,  1.2459e-01,  2.6142e-01,  3.8393e-02,  1.4018e-01,
        5.7285e-01, -5.0449e-01,  1.5094e-01,  7.3356e-02,  2.3308e-01,
       -1.6048e-01, -5.1184e-01, -1.4028e-01, -3.0110e-01, -4.3446e-01,
       -2.8257e-02,  1.9100e-03,  6.5157e-01,  1.4855e-01, -3.7255e-01,
       -4.6619e-01, -1.7223e-01, -5.1794e-01, -2.5453e-01, -2.6785e-01,
       -6.7776e-02, -3.0085e-01, -3.3212e-01, -1.5862e-01, -1.2336e-01,
       -3.8935e-01, -3.5551e-01, -3.6182e-01,  2.3197e-02, -1.7486e-01,
        2.2345e-01,  6.7557e-01,  9.6939e-03, -9.5640e-02,  1.2073e-01,
       -1.2016e-01, -1.6161e-01,  1.0661e-01, -4.5233e-01,  4.5991e-01,
        1.0367e-02, -1.0842e-01, -3.6163e-01,  2.5105e-01,  9.5780e-03,
       -6.1363e-01,  2.7494e-01,  2.1783e-01, -1.0663e-01, -9.5146e-03,
        2.7570e-01,  2.5888e-02, -5.6749e-02,  6.7667e-03,  5.3735e-01,
       -1.2224e-01,  8.8448e-02,  4.6094e-01,  4.6881e-02, -6.6557e-01,
       -3.9493e-01,  2.7362e-01,  1.2302e-01,  2.8594e-01, -2.52

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [165]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

In [166]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [167]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

True

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [168]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), 
                                                  vec(x).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [169]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

['tennis',
 'coach',
 'game',
 'teams',
 'junior',
 'Junior',
 'Team',
 'school',
 'boys',
 'leagues']

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [170]:
# halfway between day and night
spacy_closest(tokens, np.array(meanv([vec("day"), vec("night")])))

['night',
 'Day',
 'day',
 'evening',
 'Evening',
 'Morning',
 'morning',
 'afternoon',
 'nights',
 'Nights']

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [171]:
spacy_closest(tokens, vec("wine"))

['wine',
 'beer',
 'bottle',
 'Drink',
 'drink',
 'cellar',
 'fruit',
 'bottles',
 'brandy',
 'taste']

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [172]:
spacy_closest(tokens, np.array(subtractv(vec("wine"), vec("alcohol"))))

['wine',
 'cellar',
 'exquisite',
 'fabulous',
 'splendid',
 'magnificent',
 'delightful',
 'Dinner',
 'dinner',
 'sparkling']

The closest words to "water":

In [173]:
spacy_closest(tokens, vec("water"))

['water',
 'waters',
 'Salt',
 'salt',
 'pond',
 'dry',
 'liquid',
 'ocean',
 'boiling',
 'heat']

But if you add "frozen" to "water," you get "ice":

In [174]:
spacy_closest(tokens, np.array(addv(vec("water"), vec("frozen"))))

['water',
 'cold',
 'ice',
 'Salt',
 'salt',
 'dry',
 'fresh',
 'liquid',
 'boiling',
 'milk']

You can even do analogies! For example, the words most similar to "grass":

In [175]:
spacy_closest(tokens, vec("grass"))

['grass',
 'lawn',
 'trees',
 'greens',
 'grassy',
 'GARDEN',
 'garden',
 'sand',
 'foliage',
 'tree']

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [176]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, np.array(addv(blue_to_sky, vec("grass"))))

['grass',
 'Green',
 'GREEN',
 'green',
 'yellow',
 'red',
 'Red',
 'purple',
 'lawn',
 'pink']

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.